<a href="https://colab.research.google.com/github/gbessardon/Land_Cover_comparison/blob/main/CORINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running environment options

In [30]:
colab=1 # 1 running on google collab
drive=1 # 1 need to mount google drive

## LUCAS file location

In [31]:
fnshp='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_polygons.shp'
fncsv='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_attributes.csv'

## Corine file location

In [32]:
fncorine='/content/drive/MyDrive/CORINE/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif'
Corinefolder=fncorine.replace('/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif','')

# Run on collab and drive 
set colab=1 and drive=1  in running enviroment options

## MOUNT google drive

In [33]:
if drive==1:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## install geopandas and pyrpoj for collab

In [34]:
if colab==1:
  !pip install geopandas
  !pip install pyproj
  !pip install rioxarray

# import libraires

In [35]:
import os
from osgeo import gdal
from shapely import geometry
import geopandas as gpd
import numpy as np
import shutil

# Create functions

## create directory to store the segment before zipping

In [36]:
def create_sgement_dir(path=os.getcwd(),ending='LUCAS_Segment'):
    outputdir=os.path.join(path,ending)
    if not os.path.isdir(outputdir):
      os.mkdir(outputdir)
    return outputdir

## CUT Corine segments

In [47]:
def cutCORINE(fncorine,bounds,outfn,res=10):
  #Crop to a sligthly larger area
  lonmin=bounds[0]-0.1
  latmin=bounds[1]-0.1
  lonmax=bounds[2]+0.1
  latmax=bounds[3]+0.1
   # Sentinel files resolution
  src_fn=fncorine
  temp_fn='CORINEcut10m.tif' # 10m cutted filename
  ds=gdal.Open(src_fn)
  gdal.Translate(temp_fn,ds,projWin = [lonmin, latmax, lonmax, latmin],
                 projWinSRS='EPSG:4326',xRes=10,yRes=10) 
  # Recrop to the actual area and increase the resolution  
  opt=gdal.WarpOptions(dstSRS="EPSG:4326",outputBounds=bounds)
  gdal.Warp(outfn,temp_fn,options=opt)
  return

# Main

## Open LUCAS shapefile in a geodatabase

In [38]:
LUCASgdf = gpd.read_file(fnshp)

## Create directory to store the segment before zipping

In [39]:
outputdir=create_sgement_dir(path=os.getcwd(),ending='LUCAS_Segment')

## Create CORINE segments corresponding to the LUCAS segments

In [ ]:
for i,d in LUCASgdf.iterrows():
      lucasp=LUCASgdf['geometry'].loc[i]
      bounds=lucasp.bounds
      lucasid=LUCASgdf['POINT_ID'].loc[i]
      outfile=os.path.join(outputdir,str(lucasid)+'.tif')
      cutCORINE(fncorine,bounds,outfile,res=10)

## Save in a zip archive in google drive (prevents I/O errors)

In [ ]:
shutil.make_archive(os.path.join(Corinefolder,'LUCAS_Segment'), 'zip', outputdir)